# Week 5. Take-Home Project
### 應用 TF-IDF 原理找出消費者的代表商品
我們可以應用 TF-IDF 的原理去調整購買紀錄：將大家普遍都會購買的商品比重降低，消費者個人特別的選購商品比重調高。藉此我們就可以找出個體之間的差異，進一步了解客戶的消費習慣與生活風格，為行銷與推薦系統提供更多資訊。

In [1]:
from functools import reduce
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd

In [4]:
data = pd.read_csv('../91APPdataset/Orders.csv')

C:\Users\TEMP\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5495276 entries, 0 to 5495275
Data columns (total 28 columns):
DateId                         5495276 non-null int64
MemberId                       5495276 non-null int64
OrderGroupCode                 5495276 non-null object
TrackSourceTypeDef             5495276 non-null object
TrackDeviceTypeDef             5495276 non-null object
PayProfileTypeDef              5495276 non-null object
SalesOrderSlaveId              5495276 non-null int64
SalePageId                     5495276 non-null int64
IsMajor                        5495276 non-null bool
IsGift                         5495276 non-null bool
IsSalePageGift                 5495276 non-null bool
Quantity                       5495276 non-null int64
UnitPrice                      5495276 non-null float64
PromotionDiscount              5495276 non-null float64
ECouponId                      5495276 non-null int64
ECouponDiscount                5495276 non-null float64
SalesOrderSlaveT

In [4]:
len(data.MemberId.unique())

563457

### 長資料 (Long Data) 與寬資料 (Wide Data)：Pandas 的樞紐分析表函式
我們要將每個人買了什麼東西的資訊蒐集起來，並羅列出他們的數量，用`groupby()`就可以輕鬆做到。

In [5]:
member_grouped = data.groupby(['MemberId', 'SalePageId'])
long_sample = member_grouped['Quantity'].sum().head(12).reset_index()
long_sample

,MemberId,SalePageId,Quantity
0,1326,2294442,1
1,1329,1119492,1
2,1329,1413478,1
3,1329,1438703,1
4,1334,1597130,1
5,1334,1674525,1
6,1334,1883587,1
7,1336,1959833,3
8,1336,1959927,4
9,1336,1974625,10


這不是我們想要的表格。這樣的表格表示方式就是長資料(long data)，將所有特徵以 multi-index 描述，所有維度擠在左邊。可以發現，這樣的表達方式跟 json 和 xml 很相似，都是用一層一層的屬性紀錄資訊。

但是我們不能用這個操作 tf-idf，我們需要個人在橫軸(或縱軸)、物品在另一軸、內容為物品購買數量的表格，像下面這樣：

In [6]:
pd.pivot_table(long_sample,
               values='Quantity',
               index='MemberId',
               columns='SalePageId',
               fill_value=0)

SalePageId,1119492,1413478,1438703,1597130,1674525,1883587,1959833,1959927,1974625,2036844,2104438,2294442
MemberId,,,,,,,,,,,,
1326,0,0,0,0,0,0,0,0,0,0,0,1
1329,1,1,1,0,0,0,0,0,0,0,0,0
1334,0,0,0,1,1,1,0,0,0,0,0,0
1336,0,0,0,0,0,0,3,4,10,7,3,0


這樣的方式是寬資料(wide data)，可以用`pivot_table()`這個函式來完成。事實上這就是 Excel 中的樞紐分析，你可以隨意選兩個特徵，用這二個維度把資料分群，然後找出你感興趣的各群統計量。有時候這也稱 cross-tabulation。Stata 裡面的 `tab`，R 的 `tapply`，都提供了相同的功能。

StackOverflow 的文章 "[How to pivot a dataframe?](https://stackoverflow.com/questions/47152691/how-to-pivot-a-dataframe)" 提供了非常完善的說明。 

這邊我們使用約略1/5的資料運算，無法使用全部資料的瓶頸於最後敘述。

In [7]:
wide_data = data.iloc[0:1000000].groupby(['MemberId', 'SalePageId'])['Quantity'].sum().unstack(fill_value=0)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()
tfidf_gen = transformer.fit_transform(wide_data)
df_tfidf = pd.DataFrame(tfidf_gen.toarray(), columns=wide_data.columns)

In [ ]:
wide_data[wide_data>0].head()

### 使用全部紀錄遭遇到的瓶頸
猜測發生了記憶體不足與溢位的問題，導致流程無法順利進行。
參考：
1. [ValueError and IndexError for pivot_table #10582](https://github.com/pandas-dev/pandas/issues/10582)
2. [integer overflow in pandas.DataFrame.unstack()](https://github.com/pandas-dev/pandas/issues/2386)

In [ ]:
# Approach-1. 4/5 of the data
wide_data = data.iloc[0:4000000].groupby(['MemberId', 'SalePageId'])['Quantity'].sum().unstack(fill_value=0)

In [10]:
# Approach-1. whole dataset
wide_data = data.groupby(['MemberId', 'SalePageId'])['Quantity'].sum().unstack(fill_value=0)

IndexError: index 1034478353 is out of bounds for axis 0 with size 1034474425

In [22]:
# Approach-2.
data_grouped = data.groupby(['MemberId', 'SalePageId'])['Quantity'].sum().reset_index()
res = data_grouped.set_index(['MemberId','SalePageId'])['Quantity'].repeat(data_grouped['Quantity']).reset_index()

i, r = pd.factorize(res['MemberId'].values)
j, c = pd.factorize(res['SalePageId'].values)
n, m = r.size, c.size
b = np.bincount(i * m + j, minlength=n * m).reshape(n, m)
pd.DataFrame(b, r, c)

MemoryError: 